# Chapter 2. Patterns for cleaner Python

## 2.1 Assertions

In [8]:
#| error: false
assert 1==2, "error, buddy"

AssertionError: error, buddy

### Caveats with assertions

__Caveat 1__ <br>
The biggest caveat with using asserts in Python is that assertions can be globally disabled3 with the -O and -OO command line switches, as well as the PYTHONOPTIMIZE environment variable in CPython. This means that we should not use asserts for data validation:

```python
def delete_product(prod_id, user):
    assert user.is_admin(), 'Must be admin'
    assert store.has_product(prod_id), 'Unknown product'
    store.get_product(prod_id).delete()

```

If assertions are disabled, those two lines above will not run, which can lead to malicious attacks and bugs.

__Caveat 2__. Assertions s that never fail. If you pass a tuple to an assert statement, it leads to the assert condition always being true—which in turn leads to the above assert statement being useless because it can never fail and trigger an exception.

```python
assert (
    counter == 10,
    'It should have counted all the items'
)

```

In [9]:
assert (1==2, "error, buddy")

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/tmp/ipykernel_4680/257202358.py:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (1==2, "error, buddy")


## 2.2 Context Managers and the `with` Statements

To use a context manager all you need to do is add `__enter__` and `__exit__` methods to an object. Python will call these two methods at the appropriate times in the resource management cycle.


In [21]:
class ManagedFIle:
    def __init__(self, name):
        self.name = name
    def __enter__(self):
        self.file = open(self.file, 'w')
    def __exit__(self):
        if self.file:
            self.file.close()

In [22]:
from contextlib import contextmanager

In [24]:
@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f
    finally:
        f.close()

In [29]:
with managed_file('hello.txt')  as f:
    f.write('hello, world!\n')
    f.write('bye now')

In [53]:
class Indenter:
    def __init__(self):
        self.level = 0
    
    def change_level(self, value):
        self.level += value

    def print(self, text):
        print('    '*self.level + text)
   
    def __enter__(self):
        self.change_level(1)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.change_level(-1)

In [54]:
with Indenter() as indent:
    indent.print('hi!')
    with indent:
        indent.print('hello')
        with indent:
            indent.print('bonjour')
    indent.print('hey')

    hi!
        hello
            bonjour
    hey


## 2.3 Underscores, Dunders, and More

In a nutshell:
- single leading underscore is to define internal names, like _internal
- single trailing underscore is to define a name that matches Pythin keyword, like var_
- double leading underscore is _name mangling_ for Python interpreter to avoid the variable from being overriden in subclasses. See below
- double leading and trailing underscore are reserved for special "magic" Python methods
- single underscore can be used as a temporasry placeholder for a variable that is not used


:::{.callout-note}
Note, that by default Python `from bla import *` does not import names defined with a leading underscore, i.e. _internal
:::

In [59]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 12
        self.__bza = 13
t = Test()
dir(t)

['_Test__bza',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

In [60]:
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'
t2 = ExtendedTest()
t2.foo, t2._bar, t2.__baz

AttributeError: 'ExtendedTest' object has no attribute '__baz'

## 2.4 String Formatting